# Discogs Functions

In [4]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import discogsUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists
from artist import artist
from albums import albums
from album import album, albumURLInfo
from time import sleep

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-09-28 21:04:30.696523


In [2]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
art  = artist()
albs = albums(disc)
alb  = album()
dutils = discogsUtils()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/base exists
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/base-db exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-extra exists
/Volumes/Music/Discog/search exists
/Volumes/Music/Discog/search-artists exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists
Found 100 artist DB files and that is equal to the max mod value


# Artist Section

# Parse Artists (Parallel)

In [ ]:
for modVal in range(50):
    parseArtists(modVal)

In [ ]:
%load_ext autoreload
%autoreload

from artist import artist
from artists import artists
arts = artists(disc)

def parseArtists(modVal, force=False, doExtra=False):
    #arts.parseArtistModValFiles(modVal, force=force)
    arts.parseArtistModValExtraFiles(modVal)


def parseArtistsParallel(nProc=3, force=False):
    pool = Pool(processes=nProcs)
    result = pool.map_async(parseArtists, range(100)) 
    start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
    while not result.ready():
        if force is True:
            time.sleep(10)
        else:
            time.sleep(1)
    print("")
    
    elapsed(start, cmt)
    return result.get()

start,cmt=clock("Parsing All Artists")
start2,cmt2=clock("Parallel Artist Parsing")
#parseArtistsParallel()
elapsed(start2, cmt2)
#arts.buildMetadata(force=False, doAlbums=False)
elapsed(start, cmt)

# Download Artist Albums

In [ ]:
disc.getDiagnosticAlbumIDs()

In [8]:
%load_ext autoreload
%autoreload
from albums import albums
disc = discogs()
albs = albums(disc)
albs.downloadAlbumsFromArtists(maxAlbums=2, mediaTypes=None, rand=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/base exists
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/base-db exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-extra exists
/Volumes/Music/Discog/search exists
/Volumes/Music/Discog/search-artists exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists
Found 100 artist DB files and that is equal to the max mod value
Loading /Volumes/Music/Discog/artists-db/14-DB.p... Found 5061 Artists
Loading data from /Volumes/Music/Discog/diagnostic/albumKnownIDs.p
  --> This file is 425.8kB.
Loading /Volumes/Music/Discog/diagnostic/albumKnownIDs.

	Downloaded 0/1 entries of type Miscellaneous
  Found album code 3974387 with 1 artists
  Saving /Volumes/Music/Discog/albums/14/1010814/3974387.p
  --> This file is 15.7kB.
  Done and sleeping...
11 / 5061 	: 2 	 101114 	 Natasha King
	Downloaded 1/5 entries of type Singles & EPs
  Found album code 3671544 with 2 artists
  Saving /Volumes/Music/Discog/albums/74/380874/3671544.p
  --> This file is 15.9kB.
  Saving /Volumes/Music/Discog/albums/14/101114/3671544.p
  --> This file is 15.9kB.
  Done and sleeping...

== Download Rate: 25 / 1.9 = 13.0

12 / 5061 	: 0 	 1012514 	 Desmond Brice
	Downloaded 0/1 entries of type Instruments & Performance
  Found album code 6357179 with 1 artists
  Saving /Volumes/Music/Discog/albums/11/2336011/6357179.p
  --> This file is 16.9kB.
  Done and sleeping...
	Downloaded 0/10 entries of type Writing & Arrangement
  Found album code 984760 with 1 artists
  Saving /Volumes/Music/Discog/albums/66/299466/984760.p
  --> This file is 19.3kB.
  Done and sleepi

  Found album code 6326243 with 1 artists
  Saving /Volumes/Music/Discog/albums/14/1022514/6326243.p
  --> This file is 15.9kB.
  Done and sleeping...
	Downloaded 0/1 entries of type Miscellaneous
  Found album code 12447824 with 2 artists
  Saving /Volumes/Music/Discog/albums/14/1022514/12447824.p
  --> This file is 16.8kB.
  Saving /Volumes/Music/Discog/albums/81/967081/12447824.p
  --> This file is 16.8kB.
  Done and sleeping...
20 / 5061 	: 0 	 1023014 	 Buddy if you like our music go to the vet
	Downloaded 0/1 entries of type Albums
  Found album code 1263647 with 1 artists
  Saving /Volumes/Music/Discog/albums/14/1023014/1263647.p
  --> This file is 15.7kB.
  Done and sleeping...

== Download Rate: 50 / 3.7 = 13.5

	Downloaded 0/11 entries of type Miscellaneous
  Found album code 1263653 with 1 artists
  Saving /Volumes/Music/Discog/albums/14/1023014/1263653.p
  --> This file is 15.1kB.
  Done and sleeping...
  Found album code 1263651 with 1 artists
  Saving /Volumes/Music/Disco

27 / 5061 	: 1 	 102714 	 DJ Spice (2)
	Downloaded 1/1 entries of type Albums
	Downloaded 0/10 entries of type Singles & EPs
  Found album code 289180 with 1 artists
  Saving /Volumes/Music/Discog/albums/14/102714/289180.p
  --> This file is 18.4kB.
  Done and sleeping...

== Download Rate: 75 / 5.3 = 14.2

  Found album code 357250 with 2 artists
  Saving /Volumes/Music/Discog/albums/74/56674/357250.p
  --> This file is 16.5kB.
  Saving /Volumes/Music/Discog/albums/14/102714/357250.p
  --> This file is 16.5kB.
  Done and sleeping...
	Downloaded 0/1 entries of type DJ Mixes
  Found album code 732173 with 1 artists
  Saving /Volumes/Music/Discog/albums/14/102714/732173.p
  --> This file is 18.4kB.
  Done and sleeping...
28 / 5061 	: 5 	 1027814 	 Dimitre Rouskov
	Downloaded 4/6 entries of type Albums
29 / 5061 	: 2 	 1029214 	 Jefferson Pitcher
	Downloaded 2/6 entries of type Albums
30 / 5061 	: 0 	 1030014 	 Will Foster (2)
	Downloaded 0/1 entries of type Singles & EPs
  Found album co

  Found album code 1056895 with 1 artists
  Saving /Volumes/Music/Discog/albums/72/83672/1056895.p
  --> This file is 17.6kB.
  Done and sleeping...
	Downloaded 0/1 entries of type Writing & Arrangement
  Found album code 1056895 with 1 artists
  Exists /Volumes/Music/Discog/albums/72/83672/1056895.p
  Done and sleeping...
	Downloaded 0/1 entries of type Production
	  Previously downloaded.
	Downloaded 0/1 entries of type Visual
	  Previously downloaded.
36 / 5061 	: 1 	 1035314 	 Défendant Notre Cause
	Downloaded 1/2 entries of type Albums
  Found album code 366464 with 1 artists
  Saving /Volumes/Music/Discog/albums/14/1035314/366464.p
  --> This file is 17.0kB.
  Done and sleeping...
	Downloaded 0/1 entries of type Singles & EPs
  Found album code 366458 with 1 artists
  Saving /Volumes/Music/Discog/albums/14/1035314/366458.p
  --> This file is 15.2kB.
  Done and sleeping...
37 / 5061 	: 1 	 1035614 	 Althea Waites
	Downloaded 1/1 entries of type Albums
38 / 5061 	: 3 	 1036614 	 Gr

...

	Downloaded 1/3 entries of type Singles & EPs
  Found album code 719369 with 1 artists
  Saving /Volumes/Music/Discog/albums/14/858314/719369.p
  --> This file is 15.2kB.
  Done and sleeping...
4575 / 5061 	: 0 	 8614 	 Future Soul Orchestra
	Downloaded 0/1 entries of type Singles & EPs
  Found album code 215669 with 1 artists
  Saving /Volumes/Music/Discog/albums/14/8614/215669.p
  --> This file is 18.1kB.
  Done and sleeping...
4576 / 5061 	: 0 	 862014 	 Bryant Munch
	Downloaded 0/7 entries of type Instruments & Performance
  Found album code 231342 with 1 artists
  Exists /Volumes/Music/Discog/albums/48/395848/231342.p
  Done and sleeping...
  Found album code 591077 with 1 artists
  Exists /Volumes/Music/Discog/albums/93/313593/591077.p
  Done and sleeping...
4577 / 5061 	: 0 	 864114 	 Andrew Kidman
	Downloaded 0/4 entries of type Albums
  Found album code 12535848 with 1 artists
  Saving /Volumes/Music/Discog/albums/14/864114/12535848.p
  --> This file is 15.9kB.
  Done and slee

  Found album code 1362746 with 1 artists
  Saving /Volumes/Music/Discog/albums/14/869014/1362746.p
  --> This file is 14.9kB.
  Done and sleeping...
	Downloaded 0/1 entries of type Compilations
  Found album code 11018666 with 1 artists
  Saving /Volumes/Music/Discog/albums/14/869014/11018666.p
  --> This file is 16.4kB.
  Done and sleeping...
4584 / 5061 	: 0 	 869814 	 Serge Lopez
	Downloaded 0/3 entries of type Albums
  Found album code 8623582 with 1 artists
  Saving /Volumes/Music/Discog/albums/14/869814/8623582.p
  --> This file is 15.8kB.
  Done and sleeping...
  Found album code 8804987 with 1 artists
  Saving /Volumes/Music/Discog/albums/14/869814/8804987.p
  --> This file is 17.7kB.
  Done and sleeping...
4585 / 5061 	: 1 	 870314 	 F.U.B.A.R. (2)
	Downloaded 0/6 entries of type Albums
  Found album code 197551 with 1 artists
  Saving /Volumes/Music/Discog/albums/14/870314/197551.p
  --> This file is 18.5kB.
  Done and sleeping...
  Found album code 2831951 with 2 artists
  

  Found album code 2292956 with 1 artists
  Saving /Volumes/Music/Discog/albums/NAN/NAN/2292956.p
  --> This file is 19.5kB.
  Done and sleeping...
  Found album code 1044356 with 1 artists
  Exists /Volumes/Music/Discog/albums/NAN/NAN/1044356.p
  Done and sleeping...
4593 / 5061 	: 73 	 873814 	 Pablo de Sarasate
	Downloaded 26/83 entries of type Albums
	Downloaded 4/26 entries of type Singles & EPs
	Downloaded 3/9 entries of type Compilations
	Downloaded 0/2 entries of type Miscellaneous
  Found album code 909555 with 3 artists
  Saving /Volumes/Music/Discog/albums/90/526590/909555.p
  --> This file is 16.5kB.
  Saving /Volumes/Music/Discog/albums/14/873814/909555.p
  --> This file is 16.5kB.
  Saving /Volumes/Music/Discog/albums/26/456926/909555.p
  --> This file is 16.5kB.
  Done and sleeping...
  Found album code 5558562 with 3 artists
  Saving /Volumes/Music/Discog/albums/14/873814/5558562.p
  --> This file is 15.8kB.
  Saving /Volumes/Music/Discog/albums/79/526579/5558562.p
  --

  Found album code 8372369 with 3 artists
  Saving /Volumes/Music/Discog/albums/55/8055/8372369.p
  --> This file is 14.7kB.
  Saving /Volumes/Music/Discog/albums/14/880514/8372369.p
  --> This file is 14.7kB.
  Saving /Volumes/Music/Discog/albums/4/734504/8372369.p
  --> This file is 14.7kB.
  Done and sleeping...
4603 / 5061 	: 0 	 881114 	 Jackie Barnett
	Downloaded 0/3 entries of type Albums
  Found album code 766934 with 2 artists
  Saving /Volumes/Music/Discog/albums/14/881114/766934.p
  --> This file is 16.9kB.
  Saving /Volumes/Music/Discog/albums/7/651407/766934.p
  --> This file is 16.9kB.
  Done and sleeping...
  Found album code 426387 with 2 artists
  Saving /Volumes/Music/Discog/albums/14/881114/426387.p
  --> This file is 15.9kB.
  Saving /Volumes/Music/Discog/albums/7/651407/426387.p
  --> This file is 15.9kB.
  Done and sleeping...
4604 / 5061 	: 0 	 88314 	 D. Kay & Epsilon
	Downloaded 0/7 entries of type Singles & EPs
  Found album code 19360 with 2 artists
  Saving 

KeyboardInterrupt: 

# Download Searched Artist

In [6]:
%load_ext autoreload
%autoreload
from albums import albums
disc = discogs()
arts = artists(disc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/base exists
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/base-db exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-extra exists
/Volumes/Music/Discog/search exists
/Volumes/Music/Discog/search-artists exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists
Found 100 artist DB files and that is equal to the max mod value


In [7]:
arts.searchDiscogForArtist("Melanie Sparks")



===================== Searching For Melanie Sparks =====================
Downloading: https://www.discogs.com/search/?q=Melanie%20Sparks&limit=250&type=artist
Found 2 artists
2 / 2 	: 7 	 https://www.discogs.com/artist/6230694-Melanie-Sparks?sort=year%2Casc&limit=500


# Download Old Artist

In [ ]:
from time import ctime
from os import stat
from os.path import getctime
from timeUtils import getDateTime
from searchUtils import findDirs, findExt

artistIDs = {}
for dirval in findDirs("/Volumes/Music/Discog/artists/"):
    print(dirval,end="\t")
    for ifile in findExt(dirval, ext=".p"):
        mtime = ctime(getctime(ifile))
        dtime = getDateTime(mtime)        
        if dtime.year < 2019:
            artistIDs[ifile] = True
            continue
            artData = arts.getData(ifile)
            ID  = artData.ID.ID
            url = artData.url.url
            artistIDs[ID] = url
    print(len(artistIDs))

In [ ]:
artistIDtoName = disc.getArtistIDToNameData()
artistIDtoRef  = disc.getArtistIDToRefData()

In [ ]:
from fileUtils import getBaseFilename

In [ ]:
for ifile,artistURL in artistIDs.items():
    artistID = getBaseFilename(ifile)
    try:
        name = artistIDtoName[artistID]
    except:
        continue
    try:
        ref = artistIDtoRef[artistID]
    except:
        continue
        
    #print(artistID,name,ref,ifile)
    #1/0
        
    if len(name) != len(name.encode()):
        print(artistID,'\t',name)
        url = arts.getArtistURL(ref)
        savename = arts.getArtistSavename(artistID)
        try:
            arts.downloadArtistURL(url=url, savename=savename, force=True, sleeptime=1.75)
        except:
            continue

In [ ]:
arts.parseArtistModValFiles(38, force=True)
#arts.buildMetadata

In [ ]:
db38 = disc.getArtistsDBModValData(38)

In [ ]:
db38['216138'].artist.get()

In [ ]:
#[db38[x].artist.get()['name'] for x in db38.keys()]
discDB = disc.getArtistIDToNameData()

In [ ]:
discDB['216138']

In [ ]:
%load_ext autoreload
%autoreload

from artists import artists
disc = discogs()
arts = artists(disc)
arts.buildMetadata(doAlbums=False, force=True)

# Download Most Common Artist Albums

In [ ]:
from collections import Counter
db = Counter(disc.getCollectionRefCountsData())
coreAlbumsRefs = disc.getArtistIDCoreAlbumRefs()
artistMetaData = disc.getAlbumArtistMetaData()
albumDB = disc.getAlbumIDToRefData()

In [ ]:
toget = {}
for item in db.most_common(1000):
    ref,cnt  = item[0],item[1]
    artistID = dutils.getArtistID(ref)
    modVal   = str(int(artistID[-2:]))
    
    try:
        genre = artistMetaData[artistID]['Genre'].most_common(1)[0][0]
    except:
        continue
    #if genre not in ["Rock", "Pop", "Hip Hop"]:
    #    continue

    if toget.get(artistID) is None:
        toget[artistID] = {}
        
    
    coreAlbums = coreAlbumsRefs.get(artistID)
    #if len(coreAlbums) % 25 == 0:
    #print("{0: <50}{1: <30}{2: <10}{3}".format(ref,genre,artistID,len(coreAlbums)))
        
    for albumID,albumRef in coreAlbums.items():
        haveIt = albumDB.get(albumID)
        #print("\t{0: <15}{1}".format(albumID,haveIt))
        if haveIt is None:
            toget[artistID][albumID] = albumRef
            
    print("{0: <15}{1: <30}{2: <5}{3: <5}".format(artistID, genre, len(toget[artistID]), len(coreAlbums)))
        
    continue
    if False:
        print(artistID,'\t',ref)
        arts.rmIDFromDB(artistID)
        url = arts.getArtistURL(ref)
        savename = arts.getArtistSavename(artistID)
        arts.downloadArtistURL(url=url, savename=savename)
        sleep(2)
    #print(artistID,item,len(coreAlbums))

In [ ]:
len(toget)

In [ ]:
known = {}

In [ ]:
# Ignore 0-311 (start with Carl Perkins)
known = getFile("ignoreAlbum.p")
for ia,(artistID,albumRefs) in enumerate(toget.items()):
    print("\n|\n| {0} \t {1}\n|\n".format(ia,artistID))
    for i,(albumID,albumRef) in enumerate(albumRefs.items()):
        url = albs.getAlbumURL(albumRef)
            
        savename = albs.getAlbumSavename(artistID, albumID)
        if isFile(savename):
            continue

        if ia != 563:
            if known.get(url) is True:
                continue

        if False:
            known[url] = True
            continue
            
        print(artistID,'\t',ia,'\t',len(toget),'\t\t',i,'/',len(albumRefs),'\t',artistID)
        print("\tGetting: {0}".format(savename))
        retval = albs.downloadAlbumURLData(url=url, savename=savename, artistID=artistID, sleeptime=1.75)
        if retval is False:
            known[url] = True
            if len(known) % 5 == 0:
                print("\n====>    There are now {0} known downloaded albums    <====\n".format(len(known)))
                saveFile(idata=known, ifile="ignoreAlbum.p")

In [ ]:
len(known)

In [ ]:
saveFile(idata=known, ifile="ignoreAlbum.p")

In [ ]:
known = getFile("ignoreAlbum.p")

In [ ]:
for artistID,artistRefs in toget.items():
    print(artistID,len(artistRefs))

In [ ]:
tmp = disc.getAlbumIDToArtistIDData()

In [ ]:
db30 = disc.getAlbumsDBModValData(44)
from collections import Counter

In [ ]:
artistMetaData = {}
for artistID,artistData in db30.items():
    print(artistID)
    artistMetaData[artistID] = {"Genre": Counter(), "Artists": Counter(), "Style": Counter()}
    for albumID,albumData in artistData.items():
        genres = albumData.profile.genre
        if not isinstance(genres, list):
            genres = [genres]
        for genre in genres:
            artistMetaData[artistID]['Genre'][genre.name] += 1
        
        artists = albumData.artist.artists
        for artist in artists:
            artistMetaData[artistID]['Artists'][artist.name] += 1
        
        styles = albumData.profile.style
        if not isinstance(styles, list):
            styles = [styles]
        for style in styles:
            artistMetaData[artistID]['Style'][style.name] += 1

In [ ]:
artistMetaData

In [ ]:
albdata = albs.getData(ifile)

In [ ]:
for k,v in albdata.get().items():
    print(k,'\t',v.get())

In [ ]:
ArtistAlbums = disc.getArtistIDCoreAlbumIDs()
AlbumIDToName = disc.getAlbumIDToNameData()

In [ ]:
from string import ascii_uppercase, digits, punctuation
vals = "{0}{1}{2}".format(ascii_uppercase, digits, punctuation)
AsciiToName = {val: [x for x in NameToIDs.keys() if (x.startswith(val) or x.startswith(val.lower()))] for val in vals}

In [ ]:
ddb = discdb(disc)
ddb.loadArtistDBs(IDToName=IDToName, NameToIDs=NameToIDs, AsciiToName=AsciiToName)
ddb.loadAlbumDBs(ArtistAlbums=ArtistAlbums, AlbumIDToName=AlbumIDToName)
ddb.findArtist("Prince", num=1)
ddb.findArtistAlbums()

In [ ]:
%load_ext autoreload
%autoreload
disc   = discogs()
arts   = artists(disc)
albs   = albums(disc)
dutils = discogsUtils()
col    = collections(disc)

for modVal in list(range(99, 50, -1)):
    albs.downloadAlbumModValData(modVal, maxAlbums=1)

In [ ]:
IDToName    = disc.getArtistIDToNameData()
IDToRef     = disc.getArtistIDToRefData()
AlbIDToName = disc.getAlbumIDToNameData()
AlbIDToRef  = disc.getAlbumIDToRefData()

IDToCoreAlbID = disc.getArtistIDCoreAlbumIDs()
IDToAlbID     = disc.getArtistIDAlbumIDs()

In [ ]:
df = DataFrame.fromIDToCoreAlbID

In [ ]:
x   = {k: ":".join(v) for k,v in IDToCoreAlbID.items()}
pdf = DataFrame.from_dict(x, orient='index')

In [ ]:
pdf

In [ ]:
from itertools import islice
def takeN(n, iterable):
    if isinstance(iterable, dict):
        "Return first n items of the iterable as a list"
        #keys = takeN(n, iterable.keys())
        keys = list(islice(iterable, n))        
        return {k: iterable[k] for k in keys}
    elif isinstance(iterable, list):
        "Return first n items of the iterable as a list"
        return list(islice(iterable, n))
    else:
        print("No idea about {0}".format(type(iterable)))
        return None


#n_items = takeN(500000, ddb.NameToIDs.keys())
#n_items = ddb.NameToIDs.keys()

In [ ]:
from itertools import islice
def takeN(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

n_items = takeN(500000, ddb.NameToIDs.keys())
n_items = ddb.NameToIDs.keys()

In [ ]:
db38 = disc.getArtistsDBModValData(38)
db38['1032538']

In [ ]:
db38['1032538'].artist.get()

In [ ]:
artistIDtoName  = disc.getArtistIDToNameData()

In [ ]:
artistIDtoName['1032538']

In [ ]:
adc = arts.getData('/Volumes/Music/Discog/artists/38/1032538.p')
adc.artist.get()

In [ ]:
#col.createCollectionsData()
#col.downloadCollections()
#col.parseCollections()
#col.mergeCollections(debug=True, force=True)
#col.createCollectionDBs(debug=True)

In [ ]:
disc.moveAlbumFilesToNewModValue(newModValue=100, oldModValue=500)

# Artist Section

In [ ]:
db = disc.getArtistNameToIDsData()

In [5]:
#arts = artists(disc)
#art.findKnownArtists(debug=True)
#art.findUnknownArtists(minVal=200, debug=True)
#art.downloadUnknownArtists(debug=True)
#arts.parseArtistFiles()
#art.searchDiscogForArtist("Melanie Sparks")
#arts.buildMetadata()

AttributeError: 'artist' object has no attribute 'searchDiscogForArtist'

# Albums Metadata

In [ ]:
artistNameToID   = disc.getArtistNameToIDData()
artistNameToIDs  = disc.getArtistNameToIDsData()
artistIDToName   = disc.getArtistIDToNameData()        
artistRefToID    = disc.getArtistRefToIDData()
artistIDToRef    = disc.getArtistIDToRefData()
artistRefToName  = disc.getArtistRefToNameData()
artistNameToRef  = disc.getArtistNameToRefData()
artistNameToRefs = disc.getArtistNameToRefsData()

In [ ]:
albumRefToName  = disc.getAlbumRefToNameData()
albumNameToRef  = disc.getAlbumNameToRefData()

artistIDCoreAlbumIDs   = disc.getArtistIDCoreAlbumNames()
artistIDAlbumIDs       = disc.getArtistIDAlbumNames()



In [ ]:
from collections import Counter
core = ["Albums"]
albumCntr = Counter()


print("{0: <4}{1: <50}{2: <15}{3: <15}{4: <12}{5: <12}{6: <12}{7: <12}{8: <12}{9: <12}".format(
    "#","File","Core IDs-IDs","IDs-IDs","Name-IDs","Name-Refs","ID-Names","ID-Refs","Ref-IDs","Ref-Names"))


for i,dbfile in enumerate(dbfiles):
    db = getFile(dbfile)
    for artistID,artistData in db.items():
        artist      = artistData['Artist']
        artistName  = dutils.getArtistName(artist)
        artistRef   = artistData['URL']
        #print("{0: <20}{1: <50}{2}".format(artistID,artist,artistRef))
        
    print("{0: <4}{1: <50}{2: <15}{3: <15}{4: <12}{5: <12}{6: <12}{7: <12}{8: <12}{9: <12}".format(
        i,dbfile,
        len(artistIDCoreAlbumIDs),len(artistIDAlbumIDs),
        len(albumNameToID),len(albumNameToRef),len(albumIDToName),
        len(albumIDToRef),len(albumRefToID),len(albumRefToName)))



In [ ]:
ifile="/Volumes/Music/Discog/artists/0/9900.p"
fdata = getHTML(getFile(ifile, version=2))
an = artistName(fdata)
an.getArtist()
#name = artistName(fdata)
#name


In [ ]:
cnts   = [x.split()[0] for x in vals.split("\n")]
styles = [" ".join(x.split()[1:]) for x in vals.split("\n")]
d2000  = dict(zip(cnts, styles))
saveFile(idata=d2000, ifile="soundtracks.p")

# Albums

In [ ]:
albs = albums(disc)
albs.parseAlbumFiles()

In [ ]:
getFile('/Volumes/Music/Discog/albums-db/NAN-DB.p')

In [ ]:
import random
albs = albums(disc)
nErrs = False
while nErrs is False:
    nErrs = True
    albs.downloadAlbumDataData(int(random.uniform(0,disc.getMaxModVal()))) #, mediaTypes=None)        
    if False:
        nErrs = False
        print("Sleeping for 60 seconds...")
        sleep(60)
#art.findKnownArtists(debug=True)
#art.findUnknownArtists(minVal=200, debug=True)
#art.downloadUnknownArtists(debug=True)
#arts.parseArtistFiles()
#art.searchDiscogForArtist("$uicideboy$")
#arts.buildMetadata()

In [ ]:
%load_ext autoreload
%autoreload

from artist import artist
from artists import artists
art = artist()
arts = artists(disc)

toget=getFile(ifile="toget3.p")
done = {}
i = 0
errs = {}
n = len(toget)
for artistID,artistRef in toget.items():
    i += 1
    ifile = arts.getArtistSavename(artistID)
    print(i,'/',n,'\t',ifile)
    info  = art.getData(ifile)
    pages = info.pages
    
    if pages.redo is True:
        url      = arts.getArtistURL(artistRef, 1)
        savename = arts.getArtistSavename(artistID, 1)
        try:
            arts.downloadArtistURL(url=url, savename=savename, force=True, debug=True)            
        except:
            errs[artistID] = artistRef
            print("\tErr with {0}".format(artistRef))

In [ ]:
db = disc.getArtistsDBModValData(38)